# 전국어린이보호구역표준데이터
전국어린이보호구역표준데이터.csv

- 시설종류별 CCTV 설치 여부 카운트 / 대수
- 광역시도별, 시절종류별 카운트
- CCTV 평균 설치대수
- 관할 경찰서별 CCTV 대수 
- 도로폭 / CCTV 설치 대수 상관도

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


pd_rawdata = pd.read_csv('전국어린이보호구역표준데이터.csv', encoding='cp949')


In [3]:
# EDA : Exploratory Data Analysis

print(pd_rawdata.columns)
column_selection = ['시설종류','관할경찰서명','관할경찰서명','CCTV설치여부','CCTV설치대수','제공기관명','보호구역도로폭']
pd_data = pd_rawdata[column_selection]
#del pd_rawdata

Index(['시설종류', '대상시설명', '소재지도로명주소', '소재지지번주소', '위도', '경도', '관리기관명', '관할경찰서명',
       'CCTV설치여부', 'CCTV설치대수', '보호구역도로폭', '데이터기준일자', '제공기관코드', '제공기관명'],
      dtype='object')


In [8]:
# EDA : Exploratory Data Analysis

In [4]:
#print(pd_data.columns)
#print(pd_data.dtypes)



print(pd_data['보호구역도로폭'].value_counts())
print(pd_data['보호구역도로폭'].isna().value_counts())
pd_data['시설종류'] = pd_data['시설종류'].astype('category')
pd_data['CCTV설치여부'] = pd_data['CCTV설치여부'].astype('category')
#            print(pd_data.dtypes)
print(pd_data.shape)
pd_data.describe(include='all')

6          1600
8          1334
7           855
10          558
12          543
           ... 
30~35         1
12~50         1
5.0~5.5       1
6~45          1
18~23         1
Name: 보호구역도로폭, Length: 528, dtype: int64
False    10684
True      4372
Name: 보호구역도로폭, dtype: int64
(15056, 7)


C:\Users\ITPS\AppData\Local\Temp\ipykernel_14120\1006815135.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_data['시설종류'] = pd_data['시설종류'].astype('category')
C:\Users\ITPS\AppData\Local\Temp\ipykernel_14120\1006815135.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_data['CCTV설치여부'] = pd_data['CCTV설치여부'].astype('category')


,시설종류,관할경찰서명,관할경찰서명,CCTV설치여부,CCTV설치대수,제공기관명,보호구역도로폭
count,15056,15056,15056,15056,11682.000000,15056,10684
unique,10,262,262,2,NaN,225,528
top,초등학교,서부경찰서,서부경찰서,Y,NaN,경상남도,6
freq,6965,299,299,11687,NaN,759,1600
mean,NaN,NaN,NaN,NaN,2.586543,NaN,NaN
std,NaN,NaN,NaN,NaN,2.937831,NaN,NaN
min,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,1.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,2.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,3.000000,NaN,NaN


In [25]:
#결측지 확인 

# print(pd_data.shape)
# print(pd_data.describe(include='all'))

for e in pd_data:
    print(e,pd_data[e].hasnans)
    if pd_data[e].hasnans:
        print(pd_data[e].isna().value_counts())

시설종류 False


AttributeError: 'DataFrame' object has no attribute 'hasnans'

In [9]:
# CCTV 설채대수 : 결측치 처리
# CCTV 설치여부  Y --> 1
# CCTV 설치여부 N --> 0


print(pd_data['CCTV설치대수'].isna().value_counts())
filter_y = pd_data['CCTV설치여부'] == 'Y'
filter_n = pd_data['CCTV설치여부'] == 'N'

pd_data.loc[filter_y,'CCTV설치대수'] = pd_data.loc[filter_y,'CCTV설치대수'].replace(np.NaN,1)
pd_data.loc[filter_n,'CCTV설치대수'] = pd_data.loc[filter_n,'CCTV설치대수'].replace(np.NaN,0)
print(pd_data['CCTV설치대수'].isna().value_counts())

False    11682
True      3374
Name: CCTV설치대수, dtype: int64
False    15056
Name: CCTV설치대수, dtype: int64


In [13]:
def myfn1(x):
    if type(x) == type(' '):
        if '~' in x:
            m = np.array(x.split('~')).astype(np.float64).mean()
            return str(m)

# 도로폭 : 오류치 처리
# '숫자a ~ 숫자b' 형태의 문자열 -> '숫자 a와 숫자  b평의평균치 np.float64 값으로 대체
y = pd_data['보호구역도로폭'].apply(myfn1)
#print(y.dtype)
#print(y.value_counts())

# 도로폭 : 결측치 처리
# 전체 도로폭 평균치로 대체
print(pd_data['보호구역도로폭'].isna().value_counts())
y = y.astype(np.float64)
y = y.replace(np.NaN, y.mean())
print(y.isna().value_counts())
pd_data['보호구역도로폭'] = y


False    10684
True      4372
Name: 보호구역도로폭, dtype: int64
False    15056
Name: 보호구역도로폭, dtype: int64


C:\Users\ITPS\AppData\Local\Temp\ipykernel_14120\1031748456.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_data['보호구역도로폭'] = y


# 시설 종류
 dtype: category
 value:
 결측지 없음
 
 # 관할 경찰서명
 dtype : obejct
 결측치 없음
 
 # CCTV 설치여부
 dtype : category
 value : 'Y,'N';
 결측치 없음
 
 # CCTV 설치대수
 
 dtype : category
 min / max : 0 / _
 결측치 (개, 처리후 결측치 없음)
 cctv 설치여부 ('Y' -> 1)
 cctv 설치여부 ('N' ->1
 
 # 보호구역 도로폭
 dtype np.float64
 min/max:
 오류치(개, 처리후 오류치 없음)
 
숫자a - 숫자b 형태의 문자열 -> 숫자a 와 숫자b의 평균치(np.float64) 값으로 대체

결측치 (개, 처리후 결측치 없음)
전체 평균값으로 대체

 

In [14]:
import sklearn

from sklearn import datasets

x = datasets.load_diabetes()
print(x)

{'data': array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
         0.01990749, -0.01764613],
       [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
        -0.06833155, -0.09220405],
       [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
         0.00286131, -0.02593034],
       ...,
       [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
        -0.04688253,  0.01549073],
       [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
         0.04452873, -0.02593034],
       [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
        -0.00422151,  0.00306441]]), 'target': array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
        69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
        68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
        87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
       259.,  53., 190., 142.,  75., 142., 155., 225.,  59., 104., 182.,
  

In [72]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
print(pd_data)
print(pd_data['보호구역도로폭'].value_counts())

g1 = pd_data.groupby('시설종류')
print(g1)
g1.count()
g1['CCTV설치대수'].sum()

       시설종류   관할경찰서명   관할경찰서명 CCTV설치여부  CCTV설치대수     제공기관명   보호구역도로폭
0       유치원    고창경찰서    고창경찰서        Y       1.0  전라북도 고창군  10.94575
1      초등학교    고창경찰서    고창경찰서        Y       2.0  전라북도 고창군  10.94575
2       유치원    고창경찰서    고창경찰서        Y       1.0  전라북도 고창군  10.94575
3      초등학교    고창경찰서    고창경찰서        Y       2.0  전라북도 고창군  10.94575
4       유치원    고창경찰서    고창경찰서        Y       1.0  전라북도 고창군  10.94575
...     ...      ...      ...      ...       ...       ...       ...
15051  초등학교    목포경찰서    목포경찰서        Y       4.0  전라남도 목포시  10.94575
15052  초등학교    목포경찰서    목포경찰서        Y       2.0  전라남도 목포시  10.94575
15053  초등학교    목포경찰서    목포경찰서        Y       3.0  전라남도 목포시  10.94575
15054  초등학교    목포경찰서    목포경찰서        Y       4.0  전라남도 목포시  10.94575
15055  초등학교  창원중부경찰서  창원중부경찰서        Y       1.0  경상남도 창원시  10.94575

[15056 rows x 7 columns]
10.94575    12421
6.00000       330
16.50000      185
6.50000       168
7.00000       162
            ...  
31.00000        1
7.35000         1
6.

시설종류
국제학교             8.0
대안학교             1.0
어린이집          4222.0
외국인교육기관          1.0
외국인학교           11.0
유치원           6684.0
초등학교         20599.0
초등학교+어린이집       39.0
특수학교           258.0
학원              66.0
Name: CCTV설치대수, dtype: float64

In [17]:
pd_data['CCTV설치대수'].mean()

2.118026036131775

In [20]:
x = pd_data['보호구역도로폭']
y = pd_data['CCTV설치대수']
np.corrcoef(x,y)

array([[ 1.       , -0.0478016],
       [-0.0478016,  1.       ]])

In [19]:
print(pd_data)

       시설종류   관할경찰서명   관할경찰서명 CCTV설치여부  CCTV설치대수     제공기관명   보호구역도로폭
0       유치원    고창경찰서    고창경찰서        Y       1.0  전라북도 고창군  10.94575
1      초등학교    고창경찰서    고창경찰서        Y       2.0  전라북도 고창군  10.94575
2       유치원    고창경찰서    고창경찰서        Y       1.0  전라북도 고창군  10.94575
3      초등학교    고창경찰서    고창경찰서        Y       2.0  전라북도 고창군  10.94575
4       유치원    고창경찰서    고창경찰서        Y       1.0  전라북도 고창군  10.94575
...     ...      ...      ...      ...       ...       ...       ...
15051  초등학교    목포경찰서    목포경찰서        Y       4.0  전라남도 목포시  10.94575
15052  초등학교    목포경찰서    목포경찰서        Y       2.0  전라남도 목포시  10.94575
15053  초등학교    목포경찰서    목포경찰서        Y       3.0  전라남도 목포시  10.94575
15054  초등학교    목포경찰서    목포경찰서        Y       4.0  전라남도 목포시  10.94575
15055  초등학교  창원중부경찰서  창원중부경찰서        Y       1.0  경상남도 창원시  10.94575

[15056 rows x 7 columns]
